In [2]:
import requests
import json

In [12]:
# a request to localhost:8080

url = "http://localhost:8080"
# url = "https://mypersonaljscloudivnad9dy-readnft.functions.fnc.fr-par.scw.cloud"

my_prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Where is Argentine?"},
]
# my_prompt = "Where is Chile?"  # Example prompt, can be replaced with any string

# Sign the message

Now that we create messages and do not directly write them to the blockchain, we need to sign them before sending them to the server.

In [ ]:
import requests
from web3 import Web3
from eth_account.messages import encode_defunct

from dotenv import load_dotenv
import os

In [5]:
load_dotenv()
WALLET_PRIVATE_KEY = os.getenv("TEST_WALLET_PRIVATE_KEY")

In [6]:
class EthApiAuth:
    def __init__(self, api_base_url, private_key):
        self.api_base_url = api_base_url
        self.w3 = Web3()
        self.account = self.w3.eth.account.from_key(private_key)

    def create_auth_payload(self, data):
        """Create minimal authentication payload with signature"""
        if not self.account:
            raise ValueError("No account set. Provide private key first.")

        message = f"Authenticate wallet: {self.account.address}"
        signature = self.sign_message(message)

        return {
            'auth': {
                'address': self.account.address,
                'signature': signature,
                'message': message  # optional, für Debugging
            },
            "data": data
        }

    def sign_message(self, message):
        """Sign message with private key"""
        if not self.account:
            raise ValueError("No account set. Provide private key first.")

        # Sign the message
        message_encoded = encode_defunct(text=message)
        signed_message = self.account.sign_message(message_encoded)

        return "0x" + signed_message.signature.hex()

    def make_authenticated_request(self, endpoint, data, method="POST"):
        """Make authenticated API request"""
        # Create auth message
        auth_data = self.create_auth_payload(data)

        # Make request
        url = f"{self.api_base_url}{endpoint}"

        if method.upper() == "POST":
            response = requests.post(url, json=auth_data)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")
        if response.status_code != 200:
            raise Exception(f"Request failed with status code {response.status_code}: {response.text}")
        return response.json()

In [37]:
# url = "http://localhost:8080"
url = "https://mypersonaljscloudivnad9dy-llm.functions.fnc.fr-par.scw.cloud"
auth = EthApiAuth(url, WALLET_PRIVATE_KEY)
prompt_data = {"prompt": my_prompt, "useDummyData": False}

# Make authenticated POST request
user_data = auth.make_authenticated_request("/", data=prompt_data, method="POST")
print("User data:", user_data)

User data: {'content': 'Argentina is a country located in the southern part of South America. It is bordered by the following countries:\n\n* Chile to the west\n* Bolivia and Paraguay to the north\n* Brazil to the northeast\n* Uruguay and the South Atlantic Ocean to the east\n* The Drake Passage to the south, which separates it from Antarctica\n\nArgentina is the second-largest country in South America, after Brazil, and the eighth-largest country in the world. Its capital and largest city is Buenos Aires.', 'usage': {'completion_tokens': 99, 'prompt_tokens': 45, 'total_tokens': 144, 'prompt_tokens_details': None}, 'model': 'meta-llama/Llama-3.3-70B-Instruct'}


# Get the leafs

let me now also test if I can get the leaves for a specific address. The tests on the localhost are running for `jeaf_history.js`. 

In [ ]:
url = "http://localhost:8080"
# url = "https://mypersonaljscloudivnad9dy-readnft.functions.fnc.fr-par.scw.cloud"

walletAddress = "0x073F26F0C3FC100E7B075C3DC3CDE0A777497D20"
# send a get request with the prompt and tokenId as query parameters
response = requests.get(
    url, params={"address": walletAddress}
)